### 1. 요리 재료 전처리  & xlsx -> csv

In [2]:
import pandas as pd

# 1. 파일 이름 정의
folder = '../data/'
file_in = folder + 'recipe_ingredient.xlsx'
file_out = folder + 'recipe_ingredient_cleaned.csv'

try:
    # 2. 파일 로드 (한글 깨짐 및 오류 방지)
    try:
        df = pd.read_excel(file_in)
    except UnicodeDecodeError:
        print("cp949 인코딩 실패, utf-8로 재시도...")
        df = pd.read_excel(file_in, encoding='utf-8')
    except Exception as e:
        print(f"읽기 오류 발생. Python 엔진으로 재시도...: {e}")
        df = pd.read_excel(file_in, encoding='cp949', engine='python', on_bad_lines='skip')


    # --- (핵심) 3. 사용자 정의 정렬 순서 생성 ---
    # '주재료' 1순위, '부재료' 2순위, '양념' 3순위로 지정
    custom_order = {
        '주재료': 1,
        '부재료': 2,
        '양념': 3
    }
    
    # '재료타입명'을 기준으로 새로운 정렬용 'sort_order' 열을 생성합니다.
    # 목록에 없는 값은 99순위로 밀려납니다.
    df['sort_order'] = df['재료타입명'].map(custom_order).fillna(99)

    # --- 4. 데이터 정렬 ---
    # 1순위: 방금 만든 'sort_order' (주재료 > 부재료 > 양념)
    # 2순위: 레시피 코드
    # 3순위: 재료순번
    df_sorted = df.sort_values(by=['레시피 코드',  '재료순번', 'sort_order'])

    # 5. 불필요한 열 제거
    # 정렬에 사용한 'sort_order' 열은 삭제합니다.
    df_sorted = df_sorted.drop(columns=['sort_order'])

    # 6. 파일 저장
    # 엑셀 호환성을 위해 'utf-8-sig'로 저장
    df_sorted.to_csv(file_out, index=False, encoding='utf-8-sig')

    print(f"파일 처리 완료! 정렬된 파일이 '{file_out}'로 저장되었습니다.")
    print("\n정렬된 데이터 상위 5개 행:")
    print(df_sorted.head())

except FileNotFoundError:
    print(f"오류: '{file_in}' 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"오류 발생: {e}")

파일 처리 완료! 정렬된 파일이 '../data/recipe_ingredient_cleaned.csv'로 저장되었습니다.

정렬된 데이터 상위 5개 행:
   레시피 코드  재료순번  재료명  재료용량  재료타입 코드 재료타입명
0       1     1    쌀    4컵  3060001   주재료
1       1     2   안심  200g  3060001   주재료
2       1     3  콩나물   20g  3060001   주재료
3       1     4  청포묵  1/2모  3060001   주재료
4       1     5  미나리   20g  3060001   주재료


In [3]:
import pandas as pd

file_path = '../data/'
file_name = 'recipe_data.csv'

df_data = pd.read_csv(
    file_path + file_name,
    encoding='cp949',
    engine='python',
    on_bad_lines='warn' 
)

df_data.head(10)

,레시피 코드 (SEQ_RECIPE),레시피 이름(한글),간략(요약) 소개,유형코드,유형분류,음식분류코드,음식분류,조리시간,칼로리,분량,난이도,재료별 분류명,가격별 분류,대표이미지 URL,상세 URL
0,195453,콩비지동그랑땡,두부대신 콩비지를 넣어 만든 동그랑땡 맛도 좋아요!,3020001,한식,3010018,부침,30분,0Kcal,3인분,보통,,"10,000원",http://file.okdab.com/recipe/14829957726840013...,http://www.okdab.com/consumer/recipe/recipeVie...
1,195428,누드김밥,건강한 재료로 가득담아 말은 누드김밥! 고기가 없어도 맛있어요!!,3020009,퓨전,3010001,밥,20분,0Kcal,3인분,보통,,"3,000원",http://file.okdab.com/recipe/14829933250580012...,http://www.okdab.com/consumer/recipe/recipeVie...
2,180363,쪽파 새우강회,새우에 쪽파를 감아 간단하게 만들지만 새우와 쪽파의 달콤한 맛과 새콤한 초고추장의 ...,3020009,퓨전,3010007,나물/생채/샐러드,20분,0Kcal,2인분,초보환영,,"5,000원",http://file.okdab.com/recipe/14829900265530011...,http://www.okdab.com/consumer/recipe/recipeVie...
3,180344,카레토마토달걀볶음밥,토마토와 달걀을 넣은 볶음밥에 카레소스를 두른 영양만점 볶음밥,3020009,퓨전,3010001,밥,30분,0Kcal,1인분,초보환영,,"3,000원",http://file.okdab.com/recipe/14829144377250001...,http://www.okdab.com/consumer/recipe/recipeVie...
4,180330,식빵고구마파이,식빵을 이용한 간단 고구마파이! 아이들 영양 간식으로 딱!,3020002,서양,3010014,빵/과자,30분,0Kcal,2인분,초보환영,,"5,000원",http://file.okdab.com/recipe/14829104368610000...,http://www.okdab.com/consumer/recipe/recipeVie...
5,120476,배추만두,배추로 만든 깜찍한 배추주머니 만두! 아삭한 배추와 영양으로 가~득 채운 보기도 좋...,3020001,한식,3010006,찜,30분,0Kcal,2인분,보통,,"10,000원",http://file.okdab.com/recipe/14823064894890034...,http://www.okdab.com/consumer/recipe/recipeVie...
6,120463,당근잎 감자전,당근 향이 나는 향긋한 당근잎 감자전,3020001,한식,3010018,부침,20분,0Kcal,2인분,보통,,"3,000원",http://file.okdab.com/recipe/14822981248350033...,http://www.okdab.com/consumer/recipe/recipeVie...
7,120441,단호박 고등어조림,달콤한 단호박으로 만든 달콤 짭짜름한 고등어조림,3020001,한식,3010003,조림,30분,0Kcal,4인분,보통,,"5,000원",http://file.okdab.com/recipe/14822241141840002...,http://www.okdab.com/consumer/recipe/recipeVie...
8,120419,나물 월남쌈,명절을 보내고 남은 나물로 알뜰하고 맛이게 만들어 먹는 나물 월남쌈!,3020009,퓨전,3010007,나물/생채/샐러드,30분,0Kcal,2인분,보통,,"5,000원",http://file.okdab.com/recipe/14806690346880013...,http://www.okdab.com/consumer/recipe/recipeVie...
9,120401,애호박 구이,어린채소와 함께 샐러드처럼 먹을 수 있는 애호박 구이,3020001,한식,3010007,나물/생채/샐러드,20분,0Kcal,2인분,초보환영,,"3,000원",http://file.okdab.com/recipe/14806656173050012...,http://www.okdab.com/consumer/recipe/recipeVie...
